In [1]:
%autosave 180
%load_ext autoreload
%autoreload 2

import matplotlib
#matplotlib.use('qtagg')
%matplotlib tk

import nest_asyncio
#%config Completer.use_jedi = False
#%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#
import sys
sys.path.append("./randman")
sys.path.append("/tungstenfs/scratch/gzenke/depomatt/")

import matplotlib.pyplot as plt
import pickle
import numpy as np
import networkx as nx
from tqdm import trange

#
from utils import *

Autosaving every 180 seconds


/tmp/ipykernel_2459536/3621506625.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# RUN ANALYSIS ON REAL [CA] DATA 

In [2]:
#######################################################################################
######### OPTIONAL - LOAD DLC BODY COORDINATES AND CONVERT TO A NPY ARRAY #############
#######################################################################################
# TODO: integrate this code into the conversion code below
if False:
    import csv
    fname = '/media/cat/4TB/donato/nathalie/DON-007050/FS3/DON-007050_20211024_TR-BSL_FS3-ACQDLC_resnet50_open_arena_white_floorSep8shuffle1_200000.csv'

    # 
    locs = load_csv(fname)
    body_feature_idx = 5*3+1
    print ("Body feature index", np.arange(body_feature_idx,body_feature_idx+2,1))
    neck = np.float32(locs[3:,body_feature_idx:body_feature_idx+2])
    print (neck.shape)

    np.save(fname[:-4]+'_locs.npy', neck)

In [3]:
############################################################# 
############### COMPUTE PARTITIONED DATA ####################
############################################################# 
root_dir = '/media/cat/4TB/donato/nathalie/DON-007050/'

session = "FS9"

# load the required filensmes
fname_csv = glob.glob(os.path.join(root_dir,
                         session,
                         "*.csv"))[0]
fname_locs = fname_csv[:-4]+'_locs.npy'
fname_bin = os.path.join(root_dir,
                         session,
                         'binarized_traces.npz')
fname_start = os.path.join(root_dir,
                           session, 
                           'start.txt')

# 
''' The get_data function computes several things
    - the most important is partition_times which contains a list 
      of 64 entries (in the case of partitioning using 10cm segments which gives 8 x 8 grid)
    - each entry is for each square in the box and contains the frame IDs of the population vectors
      that occured in that square

'''

(f_binned, 
 locs_partitioned, 
 partition_times, 
 partition, 
 box_width, 
 box_length, 
 locs_cm, 
 partition_size,
 ) = get_data(fname_locs,
                     fname_bin,
                     fname_start)


bin upphase:  (365, 36025)
start:  0
All DLC locations:  (36000, 2)
Final locations shape:  (36000, 2)
locs:  (36000, 2)
F:  (36025, 365)
(36025, 365)
locs_binned:  (36000, 2)
1257.84 1249.9927
locs_cm:  (36000, 2)
DONE


In [4]:
###########################################################################
############ COMPUTE TRANSITION MATRIX (FOR NETWORK VISUALIZATION #########
############ AND NEURAL LOCATION OF EACH FRAME ############################       
###########################################################################
randomize = False
dimensionality = 500

#
transition_matrix, neural_location = compute_transition_matrix_2D_exploration_data(partition_times,
                                                                                   f_binned,
                                                                                   dimensionality)

#

  1%|█▏                                                                                  | 504/36024 [00:00<00:07, 5031.94it/s]

average states:  (64, 365)
transitoin matrix:  (64, 64)   neural location:  (36025,)
data_triaged pca:  (36025, 365)  data triaged average:  (64, 365)
data tiraged average:  [[0.         0.         0.         ... 0.0634501  0.         0.00793126]
 [0.         0.         0.         ... 0.09389672 0.         0.05633803]
 [0.         0.         0.         ... 0.03902439 0.         0.02926829]
 ...
 [0.         0.         0.         ... 0.04347826 0.         0.        ]
 [0.         0.         0.         ... 0.00921659 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


100%|██████████████████████████████████████████████████████████████████████████████████| 36024/36024 [00:06<00:00, 5270.74it/s]


In [6]:
###############################
########## MAKE MOVIES ########
###############################

    
def animate_2D_sameaxis2(frame):
    #global ctr, fig, ax1, ax2, sizes2
    global ctr, steps, fig, ax1, ax2, sizes2,partition, sample_rate, box_width, box_length, partition_size

    #
    plt.suptitle("Time: "+str(round(ctr/sample_rate,2)).zfill(2))

    ########### TRACK SPACE #########
    ax1.clear()
    ax1=plt.subplot(1,1,1)

    draw_grid2(ax1)

    x = locs_cm[ctr,0]/partition_size
    y = locs_cm[ctr,1]/partition_size
    ax1.scatter(x,
                y,
                c='blue',
                s=200)
    ax1.set_title("Mouse location x,y "+str(round(x*partition_size,1))+ " "+str(round(y*partition_size,1)) +
                 " (cm)")

    ax1.set_xlim(0,box_width//partition_size)
    ax1.set_ylim(0,box_length//partition_size)

    ax1.set_xticks([])
    ax1.set_yticks([])

    ############################################
    ########### NEURAL STATE SPACE #############
    ############################################

    temp = neural_location[ctr]
    x = temp//(box_width//partition_size)+0.5
    y = temp%(box_length//partition_size)+0.5
    
    #
    ax1.scatter(x,
                y,
                c='red',
                s=200)

    if ctr%50==0:
        print ("ctr: ", ctr)

    #
    ctr+=1
    
        
def draw_grid2(ax):
    global ctr, steps, fig, ax1, ax2, sizes2,partition, sample_rate, box_width, box_length, partition_size

    width = 10
    for k in range(0,box_width//partition_size+1,1):
        ax.plot([k,k],
                 [0,box_width//partition_size],
                 linewidth = 2,
                 color='black')
        
    for k in range(0,box_length//partition_size+1,1):
        ax.plot([0,box_width//partition_size],
                 [k,k],
                 linewidth = 2,
                 color='black')
    
##################################################################
# need 3variables
#    parittion:  the n x n matrix containing the physical space layout
#    locs_cm: array of size T (where T is time) giving real location of animal at time T
#    transition matrix: size n*2 x n*2
#    neural_location: size T giving neural-location of animal at time T

print ("neural location: ", neural_location.shape)

#
global ctr, steps, fig, ax1, ax2, sizes2,partition, sample_rate, box_width, box_length, partition_size

#
sample_rate = 20   # nathalie/laurenz stuff is at 20FPS

###################################
######## PLOT SPATIAL LOC #########
###################################
fig = plt.figure(figsize=(10,5))
#fig = plt.figure()
ax1=plt.subplot(1,1,1)
ax1.imshow(partition)
ax1.scatter(locs_cm[0,0]//(box_width//partition_size)+0.5,
            locs_cm[0,1]%(box_width//partition_size)+0.5,
            c='blue',
            s=200)
plt.xticks([])
plt.yticks([])
plt.xlim(0,box_width//partition_size)
plt.ylim(0,box_length//partition_size)

########################################
######## PLOT NEURAL-SPACE LOC #########
########################################

#
ax2=plt.subplot(1,1,1)
plt.xlim(0,box_width//partition_size)
plt.ylim(0,box_length//partition_size)

draw_grid2(ax2)

#
n_unique = np.unique(neural_location).shape[0]
sizes2 = np.zeros(n_unique)+25

#
sizes2[30]=500

#
temp = neural_location[0]
x = temp//(box_width//partition_size)+0.5
y = temp%(box_length//partition_size)+0.5
ax2.scatter(x,
            y,
            c='red',
            s=200)

#########################################
ctr=0
####                                      pos_track.shape[0]-1
ani = animation.FuncAnimation(fig, animate_2D_sameaxis2, frames = 100 #locs_cm.shape[0]-100
                              , interval=1, 
                              repeat=True)
#
if True:
    fname = '/home/cat/nathalie_'+session+"_partition"+str(partition_size)+'cm.mp4'

    # 
    ani.save(fname, 
         fps=sample_rate)
    plt.close()
else:
    plt.show()

# -
print ("...DONE...")

neural location:  (36025,)
ctr:  0
ctr:  50



KeyboardInterrupt



ctr:  100


In [41]:
###############################################################
############# GENERATE MULTIPLE MOVIES AT ONCE ################
###############################################################
import glob
sessions = [
            "FS1",
            "FS2",
            "FS3",
            "FS4",
            "FS5",
            "FS6",
            "FS7",
            "FS8",
            "FS9",
            "FS10",
            "FS11",
            "FS12",
]

sessions = ['FS9']

#
for session in sessions:
    global ctr, steps, fig, ax1, ax2, sizes2,partition, sample_rate, box_width, box_length, partition_size

    # 
    root_dir = '/media/cat/4TB/donato/nathalie/DON-007050/'
    fname_csv = glob.glob(os.path.join(root_dir,
                             session,
                             "*.csv"))[0]
    print ("fname csv: ", fname_csv)

    fname_locs = fname_csv[:-4]+'_locs.npy'

    fname_bin = os.path.join(root_dir,
                             session,
                             'binarized_traces.npz')

    fname_start = os.path.join(root_dir,
                               session, 
                               'start.txt')
    #
    # convert_dlc(fname_csv)

    (f_binned, 
     locs_partitioned, 
     partition_times, 
     partition, 
     box_width, 
     box_length, 
     locs_cm, 
     partition_size,
     ) = get_data(fname_locs,
                         fname_bin,
                         fname_start)

    #########################################
    randomize = False
    dimensionality = 500

    #
    transition_matrix, neural_location = compute_transition_matrix_2D_exploration_data(partition_times,
                                                                                       f_binned,
                                                                                       dimensionality)

    #
    sample_rate = 20   # nathalie/laurenz stuff is at 20FPS
    
    ###########################################
    ###########################################    
    ###########################################
    
    x = neural_location//(box_width//partition_size)+0.5
    y = neural_location%(box_length//partition_size)+0.5

    x = x*10
    y = y*10

    locs_decoded = np.vstack((x,y)).T
    print (locs_decoded.shape)

    print ("locs partitioned:", locs_cm.shape)
    print ("locs_decoded: ", locs_decoded.shape)
    
    dir_loc = '/run/user/1000/gvfs/smb-share:server=unibasel.ads.unibas.ch,share=bz/RG Donato/Microscopy/Nathalie/For_Catalin_Population_Analysis_2D/'
    fname = dir_loc+'nathalie_'+session+"_locations"+str(partition_size)+'cm.npz'
    
    np.savez(fname,
            locs_cm = locs_cm,
            locs_decoded = locs_decoded)
    
    #####################################################################################
    ################################ MAKE MOVIES ########################################
    #####################################################################################

    
    ###################################
    ####### PLOT SPATIAL LOCATIONS ####
    ###################################
    
    if True:
        fig = plt.figure(figsize=(10,5))
        #fig = plt.figure()
        ax1=plt.subplot(1,1,1)
        ax1.imshow(partition)
        ax1.scatter(locs_cm[0,0]//(box_width//partition_size)+0.5,
                    locs_cm[0,1]%(box_width//partition_size)+0.5,
                    c='blue',
                    s=200)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(0,box_width//partition_size)
        plt.ylim(0,box_length//partition_size)

        ########################################
        ######## PLOT NEURAL-SPACE LOC #########
        ########################################

        #
        #ax2=plt.subplot(1,2,2)
        plt.xlim(0,box_width//partition_size)
        plt.ylim(0,box_length//partition_size)

        draw_grid(ax1)

        #
        n_unique = np.unique(neural_location).shape[0]
        sizes2 = np.zeros(n_unique)+25

        #
        sizes2[30]=500

        #
        temp = neural_location[0]
        x = temp//(box_width//partition_size)+0.5
        y = temp%(box_length//partition_size)+0.5
        ax2.scatter(x,
                    y,
                    c='red',
                    s=200)

        #########################################
        ctr=0
        ####                                      pos_track.shape[0]-1
        ani = animation.FuncAnimation(fig, animate_2D_sameaxis, frames = 5000 #locs_cm.shape[0]-100
                                      , interval=1, 
                                      repeat=True)
        #
        if True:
            #fname = '/home/cat/nathalie_'+session+"_partition"+str(partition_size)+'cm.mp4'
            #dir_loc = '/run/user/1000/gvfs/smb-share:server=unibasel.ads.unibas.ch,share=bz/RG Donato/Microscopy/Nathalie/For_Catalin_Population_Analysis_2D/'
            dir_loc = '/home/cat/'
            fname = dir_loc+'nathalie_'+session+"_partition"+str(partition_size)+'cm.mp4'

            # 
            ani.save(fname, 
                     fps=sample_rate)
            plt.close()
        else:
            plt.show()

        # -
    print ("...DONE...")



  0%|                                                                                          | 0/36024 [00:00<?, ?it/s]

fname csv:  /media/cat/4TB/donato/nathalie/DON-007050/FS9/DON-007050_20211030_TR-BSL_FS9-ACQDLC_resnet50_open_arena_white_floorSep8shuffle1_200000.csv
bin upphase:  (365, 36025)
start:  0
All DLC locations:  (36000, 2)
Final locations shape:  (36000, 2)
locs:  (36000, 2)
F:  (36025, 365)
(36025, 365)
locs_binned:  (36000, 2)
1257.84 1249.9927
locs_cm:  (36000, 2)
DONE
average states:  (64, 365)
transitoin matris:  (64, 64)   neural location:  (36025,)
data_triaged pca:  (36025, 365)  data triaged average:  (64, 365)
data tiraged average:  [[0.         0.         0.         ... 0.0634501  0.         0.00793126]
 [0.         0.         0.         ... 0.09389672 0.         0.05633803]
 [0.         0.         0.         ... 0.03902439 0.         0.02926829]
 ...
 [0.         0.         0.         ... 0.04347826 0.         0.        ]
 [0.         0.         0.         ... 0.00921659 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


100%|████████████████████████████████████████████████████████████████████████████| 36024/36024 [00:06<00:00, 5230.66it/s]


(36025, 2)
locs partitioned: (36000, 2)
locs_decoded:  (36025, 2)
ctr:  0
ctr:  50
ctr:  100
ctr:  150
ctr:  200
ctr:  250
ctr:  300
ctr:  350
ctr:  400
ctr:  450
ctr:  500
ctr:  550
ctr:  600
ctr:  650
ctr:  700
ctr:  750
ctr:  800
ctr:  850
ctr:  900
ctr:  950
ctr:  1000
ctr:  1050
ctr:  1100
ctr:  1150
ctr:  1200
ctr:  1250
ctr:  1300
ctr:  1350
ctr:  1400
ctr:  1450
ctr:  1500
ctr:  1550
ctr:  1600
ctr:  1650
ctr:  1700
ctr:  1750
ctr:  1800
ctr:  1850
ctr:  1900
ctr:  1950
ctr:  2000
ctr:  2050
ctr:  2100
ctr:  2150
ctr:  2200
ctr:  2250
ctr:  2300
ctr:  2350
ctr:  2400
ctr:  2450
ctr:  2500
ctr:  2550
ctr:  2600
ctr:  2650
ctr:  2700
ctr:  2750
ctr:  2800
ctr:  2850
ctr:  2900
ctr:  2950
ctr:  3000
ctr:  3050
ctr:  3100
ctr:  3150
ctr:  3200
ctr:  3250
ctr:  3300
ctr:  3350
ctr:  3400
ctr:  3450
ctr:  3500
ctr:  3550
ctr:  3600
ctr:  3650
ctr:  3700
ctr:  3750
ctr:  3800
ctr:  3850
ctr:  3900
ctr:  3950
ctr:  4000
ctr:  4050
ctr:  4100
ctr:  4150
ctr:  4200
ctr:  4250
ctr:  4300


In [2]:
#
def draw_grid_cm(ax, 
                 box_width,
                 box_length,
                 partition_size):
    width=10
    for k in range(0,box_width+1,width):
        ax.plot([k,k],
                 [0,box_width],
                 linewidth = 2,
                 color='grey')
        
    for k in range(0,box_length+1,width):
        ax.plot([0,box_length],
                 [k,k],
                 linewidth = 2,
                 color='grey')

if True:
    plt.figure()
    ax=plt.subplot(111)
    plt.plot(locs_cm[:,0],
             locs_cm[:,1])
    draw_grid_cm(ax, 
                 box_width,
                 box_length,
                 partition_size)


    plt.xlim(0,80)
    plt.ylim(0,80)
    plt.xlabel("cm")
    plt.ylabel("cm")
    plt.show()


NameError: name 'locs_cm' is not defined

In [26]:
print ("locs partitioned:", locs_cm.shape)
print ("locs_decoded: ", locs_decoded.shape)



locs partitioned: (36000, 2)
locs_decoded:  (36025, 2)


In [30]:
data = np.load('/media/cat/4TB/donato/nathalie_FS1_locations10cm.npz')

locs_cm = data['locs_cm']
locs_decoded = data['locs_decoded']

print (locs_cm.shape, locs_decoded.shape)

(36000, 2) (36025, 2)


In [86]:
#########################################
###### MAKE TRANSITION MATRIX ###########
#########################################
from tqdm import trange

 
########################################################
# len_seg_cm = 1       # discretize belt position into 6 ~equal sections
# triage_value = 0.25  # remove xx% most outlier states
# bin_width = 5        # population vector binning
# sample_rate = 30
randomize = False


#
dimensionality = 360

#
transition_matrix, neural_location = compute_transition_matrix_2D_exploration_data(partition_times,
                                                                                   f_binned,
                                                                                   dimensionality)
 

  1%|█▉                                                                                                                                     | 524/36024 [00:00<00:06, 5232.76it/s]

average states:  (64, 360)
transitoin matris:  (64, 64)   neural location:  (36025,)
data_triaged pca:  (36025, 360)  data triaged average:  (64, 360)
data tiraged average:  [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.29776248 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.07489598 0.         0.0221914 ]
 [0.         0.         0.00194742 ... 0.01265823 0.         0.02142162]
 [0.         0.         0.         ... 0.         0.         0.02480315]]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36024/36024 [00:06<00:00, 5381.81it/s]


In [87]:
###########################################################
###########################################################
###########################################################
# PLOT NETWORK GRAPH       
#
plt.figure()
ax=plt.subplot(1,2,1)
plt.imshow(transition_matrix)

ax=plt.subplot(1,2,2)
plot_networkx(transition_matrix,
              dimensionality)

plt.show()


In [8]:
##########################################
############# make 2D DIAGRAM ############
##########################################

# partition contains matrix of n by n
# NO NEED TO DRAW< ALREADY HAVE IT ABOVE



In [91]:
###############################
########## MAKE MOVIES ########
###############################

##################################################################
# need 3variables
#    parittion:  the n x n matrix containing the physical space layout
#    locs_cm: array of size T (where T is time) giving real location of animal at time T
#    transition matrix: size n*2 x n*2
#    neural_location: size T giving neural-location of animal at time T

print ("neural location: ", neural_location.shape)

#
global ctr, steps, fig, ax1, ax2, sizes2,partition, sample_rate, box_width, box_length, partition_size
sample_rate = 20   # nathalie/laurenz stuff is at 20FPS

###################################
######## PLOT SPATIAL LOC #########
###################################
fig = plt.figure(figsize=(10,5))
#fig = plt.figure()
ax1=plt.subplot(1,2,1)
ax1.imshow(partition)
ax1.scatter(locs_cm[0,0]//(box_width//partition_size)+0.5,
            locs_cm[0,1]%(box_width//partition_size)+0.5,
            c='blue',
            s=200)
plt.xticks([])
plt.yticks([])
plt.xlim(0,box_width//partition_size)
plt.ylim(0,box_length//partition_size)

########################################
######## PLOT NEURAL-SPACE LOC #########
########################################

#
ax2=plt.subplot(1,2,2)
plt.xlim(0,box_width//partition_size)
plt.ylim(0,box_length//partition_size)

draw_grid(ax2)

#
n_unique = np.unique(neural_location).shape[0]
sizes2 = np.zeros(n_unique)+25

#
sizes2[30]=500

#
temp = neural_location[0]
x = temp//(box_width//partition_size)+0.5
y = temp%(box_length//partition_size)+0.5
ax2.scatter(x,
            y,
            c='red',
            s=200)

#########################################
ctr=0
####                                      pos_track.shape[0]-1
ani = animation.FuncAnimation(fig, animate_2D, frames = 5000 #locs_cm.shape[0]-100
                              , interval=1, 
                              repeat=True)
#
if True:
    fname = '/home/cat/nathalie_'+session+"_partition"+str(partition_size)+'cm.mp4'

    # 
    ani.save(fname, 
         fps=sample_rate)
    plt.close()
else:
    plt.show()

# -
print ("...DONE...")


neural location:  (36025,)
ctr:  0
ctr:  50
ctr:  100
ctr:  150
ctr:  200
ctr:  250
ctr:  300
ctr:  350
ctr:  400
ctr:  450
ctr:  500
ctr:  550
ctr:  600
ctr:  650
ctr:  700
ctr:  750
ctr:  800
ctr:  850
ctr:  900
ctr:  950
ctr:  1000
ctr:  1050
ctr:  1100
ctr:  1150
ctr:  1200
ctr:  1250
ctr:  1300
ctr:  1350
ctr:  1400
ctr:  1450
ctr:  1500
ctr:  1550
ctr:  1600
ctr:  1650
ctr:  1700
ctr:  1750
ctr:  1800
ctr:  1850
ctr:  1900
ctr:  1950
ctr:  2000
ctr:  2050
ctr:  2100
ctr:  2150
ctr:  2200
ctr:  2250
ctr:  2300
ctr:  2350
ctr:  2400
ctr:  2450
ctr:  2500
ctr:  2550
ctr:  2600
ctr:  2650
ctr:  2700
ctr:  2750
ctr:  2800
ctr:  2850
ctr:  2900
ctr:  2950
ctr:  3000
ctr:  3050
ctr:  3100
ctr:  3150
ctr:  3200
ctr:  3250
ctr:  3300
ctr:  3350
ctr:  3400
ctr:  3450
ctr:  3500
ctr:  3550
ctr:  3600
ctr:  3650
ctr:  3700
ctr:  3750
ctr:  3800
ctr:  3850
ctr:  3900
ctr:  3950
ctr:  4000
ctr:  4050
ctr:  4100
ctr:  4150
ctr:  4200
ctr:  4250
ctr:  4300
ctr:  4350
ctr:  4400
ctr:  4450
ctr:  

In [77]:
print (neural_location.shape, locs_cm.shape)

temp2 = neural_location//(box_width//partition_size)
print (np.unique(temp2))
temp2 = neural_location%(box_width//partition_size)
print (np.unique(temp2))
print (partition_size)
print (np.unique(neural_location))

(36025,) (36000, 2)
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
10
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
